In [21]:
import pandas as pd
from textblob import TextBlob
from concurrent.futures import ThreadPoolExecutor

def analyze_sentiment(df):
    # Define a function to analyze sentiment for a single review
    def analyze_single_review(review_text):
        blob = TextBlob(review_text)
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        # Classify sentiment based on polarity
        if polarity > 0.1:
            sentiment_label = 'POSITIVE'
        elif polarity < -0.1:
            sentiment_label = 'NEGATIVE'
        else:
            sentiment_label = 'NEUTRAL'
        
        return sentiment_label, polarity, subjectivity

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        sentiments = list(executor.map(analyze_single_review, df['review_text'].tolist()))

    # Create separate columns for sentiment label, polarity, and subjectivity
    df['sentiment_label'] = [sentiment[0] for sentiment in sentiments]
    df['sentiment_score'] = [sentiment[1] for sentiment in sentiments]
    df['subjectivity_score'] = [sentiment[2] for sentiment in sentiments]
    
    return df

# Example usage
if __name__ == "__main__":
    # Sample data
    
    df = pd.read_csv('D:/final project/cleaned_final_project_data.csv')

    # Analyze sentiment
    analyzed_df = analyze_sentiment(df)
    print(analyzed_df)


                                            product_id  rating  \
0    Motorola Edge 50 Pro 5G with 125W Charger (Lux...       5   
1    Motorola Edge 50 Pro 5G with 125W Charger (Lux...       4   
2    Motorola Edge 50 Pro 5G with 125W Charger (Lux...       5   
3    Motorola Edge 50 Pro 5G with 125W Charger (Lux...       5   
4    Motorola Edge 50 Pro 5G with 125W Charger (Lux...       5   
..                                                 ...     ...   
437  Nothing Phone (2a) Plus (Black, 256 GB)  (12 G...       5   
438  Nothing Phone (2a) Plus (Black, 256 GB)  (12 G...       5   
439  Nothing Phone (2a) Plus (Black, 256 GB)  (12 G...       5   
440  Nothing Phone (2a) Plus (Black, 256 GB)  (12 G...       5   
441  Nothing Phone (2a) Plus (Black, 256 GB)  (12 G...       5   

                                           review_text sentiment_label  \
0    Do not believe the negative reviews. This is a...        POSITIVE   
1    Reviewing after 2days of using Moto Edge 50pro...     

In [25]:
analyzed_df.to_csv('D:/final project/analyzed_df.csv', index=False)

In [143]:
import pickle
import pandas as pd
from textblob import TextBlob

# Function to analyze sentiment for the specific feature in the user prompt
def filter_reviews_by_feature(df, feature, sentiment_type):
    df['related_sentiment'] = df['review_text'].apply(
        lambda review: TextBlob(review).sentiment.polarity if feature.lower() in review.lower() else None
    )

    # Filter based on the sentiment type
    if sentiment_type == 'positive':
        df_filtered = df[df['related_sentiment'] > 0].copy()
    elif sentiment_type == 'negative':
        df_filtered = df[df['related_sentiment'] < 0].copy()
    else:
        df_filtered = df[(df['related_sentiment'] >= -0.1) & (df['related_sentiment'] <= 0.1)].copy()

    df_filtered['abs_sentiment'] = df_filtered['related_sentiment'].abs()
    
    return df_filtered

# Function to extract features and sentiment from the user prompt
def extract_features_from_prompt(user_prompt):
    features = ['camera', 'battery', 'display', 'performance', 'design', 'mobile', 'phone']
    positive_phrases = ['best', 'good', 'high', 'quality', 'excellent']
    negative_phrases = ['worst', 'bad', 'poor', 'low']
    neutral_phrases = ['average', 'medium', 'normal']
    
    feature_sentiment = {}
    
    for feature in features:
        if feature in user_prompt.lower():
            positive = any(phrase in user_prompt.lower() for phrase in positive_phrases)
            negative = any(phrase in user_prompt.lower() for phrase in negative_phrases)
            neutral = any(phrase in user_prompt.lower() for phrase in neutral_phrases)
            
            if positive:
                feature_sentiment[feature] = 'positive'
            elif negative:
                feature_sentiment[feature] = 'negative'
            elif neutral:
                feature_sentiment[feature] = 'neutral'

    return feature_sentiment

# Function to rank mobiles based on feature sentiment
def rank_mobiles(df, feature, sentiment_type, top_n=3):
    df_filtered = filter_reviews_by_feature(df, feature, sentiment_type)

    if sentiment_type == 'positive':
        rankings = df_filtered.groupby('product_id').agg(
            average_sentiment=('related_sentiment', 'mean')
        ).reset_index()
        sorted_rankings = rankings.sort_values(by='average_sentiment', ascending=False)
    elif sentiment_type == 'negative':
        rankings = df_filtered.groupby('product_id').agg(
            average_sentiment=('abs_sentiment', 'mean')
        ).reset_index()
        sorted_rankings = rankings.sort_values(by='average_sentiment', ascending=True)
    else:
        rankings = df_filtered.groupby('product_id').agg(
            average_sentiment=('related_sentiment', 'mean')
        ).reset_index()
        sorted_rankings = rankings.sort_values(by='average_sentiment', ascending=False)

    top_mobiles = sorted_rankings.head(top_n)['product_id'].tolist()

    return top_mobiles

# Function to rank mobiles by multiple features
def rank_mobiles_by_multiple_features(df, feature_sentiment_map, top_n=3):
    combined_rankings = pd.DataFrame()

    for feature, sentiment_type in feature_sentiment_map.items():
        feature_rankings = filter_reviews_by_feature(df, feature, sentiment_type)
        
        feature_rankings = feature_rankings.groupby('product_id').agg(
            average_sentiment=(f'related_sentiment', 'mean')
        ).reset_index()

        if combined_rankings.empty:
            combined_rankings = feature_rankings
        else:
            combined_rankings = pd.merge(combined_rankings, feature_rankings, on='product_id', suffixes=('', f'_{feature}'))

    combined_rankings['total_score'] = combined_rankings.filter(like='average_sentiment').sum(axis=1)

    sorted_combined_rankings = combined_rankings.sort_values(by='total_score', ascending=False)

    top_combined_mobiles = sorted_combined_rankings.head(top_n)['product_id'].tolist()

    return top_combined_mobiles

# Function to generate recommendations
def generate_recommendations(df, user_prompt, top_n=3):
    feature_sentiment = extract_features_from_prompt(user_prompt)

    if len(feature_sentiment) == 1:
        feature, sentiment_type = next(iter(feature_sentiment.items()))
        ranked_mobiles = rank_mobiles(df, feature, sentiment_type, top_n)
    else:
        ranked_mobiles = rank_mobiles_by_multiple_features(df, feature_sentiment, top_n)

    response = "\n".join([f"Rank {i+1}: {mobile}" for i, mobile in enumerate(ranked_mobiles)])

    return response

# Function to save the model and data
def save_model(dataframe, file_name="mobile_recommendation_model.pkl"):
    with open(file_name, 'wb') as file:
        pickle.dump(dataframe, file)
    print(f"Model saved to {file_name}")

# Function to load the model
def load_model(file_name="mobile_recommendation_model.pkl"):
    with open(file_name, 'rb') as file:
        dataframe = pickle.load(file)
    return dataframe

# Example usage
if __name__ == "__main__":
    # Load your CSV data
    df = pd.read_csv('D:/final project/analyzed_df.csv')

    # Save the DataFrame (you could save the full model in more advanced setups)
    save_model(df, 'mobile_recommendation_model.pkl')

    # Test user prompt
    user_prompt = input("Enter your mobile preferences (e.g., 'I need the best camera and average battery life'): ")

    # Generate recommendations based on the prompt
    recommendations = generate_recommendations(df, user_prompt)

    # Later, you can load the saved model and use it
    df_loaded = load_model('mobile_recommendation_model.pkl')
    recommendations_loaded = generate_recommendations(df_loaded, user_prompt)
    print("Recommendations from loaded model:")
    print(recommendations_loaded)



Model saved to mobile_recommendation_model.pkl


Enter your mobile preferences (e.g., 'I need the best camera and average battery life'):  i need best batter life phone


Recommendations from loaded model:
Rank 1: OPPO F27 Pro+ (Dusk Pink, 256 GB)  (8 GB RAM)
Rank 2: Nothing Phone (2a) Plus (Black, 256 GB)  (12 GB RAM)
Rank 3: SAMSUNG Galaxy S23 FE (Mint, 128 GB)  (8 GB RAM)
